In [ ]:
import numpy as np
import os
import networkx as nx
from nilearn import datasets, plotting
from tqdm import tqdm
from nilearn.connectome import ConnectivityMeasure
import numpy as np
from jax import numpy as jnp
import jraph
import yaml
from src.plots import plot_graph, plot_regions
from src.utils import ROIS, DATA_DIR, SUBJECTS

## fmri

In [ ]:
import os
def subject_dir_files(subject):
    return [
        os.path.join(DATA_DIR, subject, "roi_masks", f)
        for f in sorted(os.listdir(os.path.join(DATA_DIR, subject, "roi_masks")))
        if f.startswith("mapping_")
    ]

## connectome

In [ ]:
subject = "subj01"
lh_fmri, rh_fmri = get_fmri(subject)
# res = sum(map(lambda x: fsaverage_roi_response_to_image(subject, x, 0, "left"), rois))
plot_regions(subject, rois, 'left', 0)

In [ ]:
def connectome_from_roi_response(subject, roi, hem):  # this is wrong
    roi_mask = get_roi_mask(subject, roi, hem, atlas="challenge")
    fmri = lh_fmri if hem == "left" else rh_fmri
    roi_response = fmri[:, roi_mask]
    connectivity_measure = ConnectivityMeasure(kind="covariance")
    connectivity_matrix = connectivity_measure.fit_transform([roi_response])[0]
    connectome = connectivity_matrix_to_connectome(connectivity_matrix)
    return connectome


def connectivity_matrix_to_connectome(connectivity_matrix):
    # given a connectivity matrix, return a graph
    N = connectivity_matrix.shape[0]
    thresh = np.percentile(
        np.abs(connectivity_matrix), 100 * (N - (N / 100)) / N
    )  # consider thresholding differently as n edges increases with nodes ** 2
    connectivity_matrix[np.abs(connectivity_matrix) < thresh] = 0
    # set diagonal to 0
    np.fill_diagonal(connectivity_matrix, 0)
    graph = nx.from_numpy_array(connectivity_matrix)
    return graph, connectivity_matrix

In [ ]:
def build_connectome(roi, hem):
    nx_graph, _ = connectome_from_roi_response(roi, hem)
    # nx_graph = nx.karate_club_graph()  # TODO: replace with real graph
    nodes = jnp.eye(len(nx_graph.nodes))
    edges = None  # jnp.ones((len(G.edges), 1))
    senders = jnp.array([e[0] for e in list(nx_graph.edges)])
    receivers = jnp.array([e[1] for e in list(nx_graph.edges)])
    n_node = len(nodes)
    n_edge = len(senders)
    global_context = None  # jnp.array([1.0])
    graph = jraph.GraphsTuple(
        nodes=nodes,
        edges=edges,
        senders=senders,
        receivers=receivers,
        n_node=n_node,
        n_edge=n_edge,
        globals=global_context,
    )
    return graph


graph = build_connectome("V1d", "left")